In [1]:
import simpy
import random
import numpy as np

In [2]:
seed = 978
random.seed(seed)

# Arrival mean                              --EXPONENTIAL
interarrival_mean = 6 

# Operator 1 mean and std for service time  --LOGNORMAL
m = 12 
s = 6
M = np.log(m**2/np.sqrt(m**2+s**2))
S = np.log((m**2+s**2)/m**2)

# Operator 2 service time range             --UNIFORM
service_range = [1,7]

# Voice recognition mean -                  --EXPONENTIAL
router_mean = 5

In [3]:
router_customer = []
opr_1_break = 0
opr_2_break = 0
opr_1_break_times = []
opr_2_break_times = []
shift_started = True
time_operator_1 = False
time_operator_2 = False

In [4]:
def service(env, opr):
    if opr==operator1:
        yield env.timeout(random.lognormvariate(M,S))
        
    elif opr==operator2:
        yield env.timeout(random.uniform(*service_range))

In [5]:
class Customer(object):
    def __init__(self, name, env):
        self.env = env
        self.name = name
        self.arrival_t = self.env.now
        self.action = env.process(self.call())
        
    
    def call(self):
        global time_operator_1
        print(opr_1_break)
        print(opr_2_break)
        print('%s initiated a call at %g' % (self.name, self.env.now))
        # Voice recognition and routing
        router_customer.append(self.name)
        yield env.timeout(random.expovariate(1/router_mean))
        if random.uniform(0,1) < .3:
            self.operator = operator1
        else:
            self.operator = operator2
            
        router_customer.remove(self.name)
            
        # Voice recognition failure
        if random.uniform(0,1) < .1:
            print('%s hangs up the call at %g' % (self.name, self.env.now))
            return 
        
        print('%s is rooted to operator %d at %g' % (self.name,(self.operator==operator2)+1, self.env.now))
        
        with self.operator.request() as req:
        
            result = yield req | env.timeout(10)
            # Reneging for 10 mins
            if req not in result:
                print('%s is left the operator %d after 10 minutes at %g' % (self.name,
                                                               (self.operator==operator2)+1,
                                                               self.env.now))
                return 
            print('%s is assigned to the operator %d at %g' % (self.name,
                                                               (self.operator==operator2)+1,
                                                               self.env.now))
            yield self.env.process(service(env,self.operator))
            print('%s is done at %g' % (self.name, self.env.now))
        
       

In [6]:
def is_available(opr):
    return opr.count == 0

In [7]:
#Determining breaks
def break_number():
    return np.random.poisson(8, size=2)

In [8]:
def insertionSort(operator_list):
    for i in range(1,len(operator_list)):
        val = operator_list[i]
        pos = i
        while pos>0 and operator_list[pos-1]>val:
            operator_list[pos]=operator_list[pos-1]
            pos = pos-1
        operator_list[pos]=val
    print(operator_list)

In [9]:
#Break request times and sorting
def break_time(opr_1_break, opr_2_break):
    for k in range(opr_1_break):
        opr_1_break_times.append(random.uniform(0,480))
    for k in range(opr_2_break):
        opr_2_break_times.append(random.uniform(0,480))
    insertionSort(opr_1_break_times)
    insertionSort(opr_2_break_times)

In [10]:
def shift(env):
    global shift_started
    yield env.timeout(480)
    shift_started = True

In [11]:
def customer_generator(env):
    global opr_1_break
    global opr_2_break
    global shift_started
    global time_operator_1
    global time_operator_2
    j = 0
    breaks_1 = 0
    breaks_2 = 0
    for i in range(1000):
        #To determine whether the shift is started, a process is opened.
        #Every 480 a new shift starts.
        if(shift_started):
            [opr_1_break, opr_2_break] = break_number()
            break_time(opr_1_break, opr_2_break)
            breaks_1 = 0
            breaks_2 = 0
            j += 1
            env.process(shift(env))
            shift_started = False
        
        yield env.timeout(random.expovariate(1/interarrival_mean))
        
        # Voice recognition system limit
        if len(router_customer) < 100:
            customer = Customer('Customer %s' %(i+1), env)
        

In [12]:
env = simpy.Environment()
operator1 = simpy.Resource(env, capacity = 1)
operator2 = simpy.Resource(env, capacity = 1)
env.process(customer_generator(env))
env.run() 

[44.63307524333047, 74.39519318760645, 209.12059240981336, 226.6161423659573, 255.9664319411964, 292.4094909717117, 385.08427703724, 387.8161389706391, 397.1018255104171, 474.21742412177036]
[34.07604646614267, 90.0478027519914, 109.63388372306741, 126.55138170720205, 174.5488473525917, 186.33400836362188, 250.41985237236503, 349.81705835821435, 361.7433553998562, 393.61533834425836, 407.47900618656155, 411.4586150925984, 414.16006451794135, 434.4019150966053, 467.5377281534712]
10
15
Customer 1 initiated a call at 0.225893
Customer 1 is rooted to operator 2 at 2.77477
Customer 1 is assigned to the operator 2 at 2.77477
Customer 1 is done at 4.87612
10
15
Customer 2 initiated a call at 8.03779
Customer 2 is rooted to operator 1 at 12.0286
Customer 2 is assigned to the operator 1 at 12.0286
10
15
Customer 3 initiated a call at 16.9632
Customer 3 is rooted to operator 2 at 22.0669
Customer 3 is assigned to the operator 2 at 22.0669
10
15
Customer 4 initiated a call at 22.1613
10
15
Custo

Customer 280 initiated a call at 1758.71
10
5
Customer 281 initiated a call at 1760.07
10
5
Customer 282 initiated a call at 1763.8
Customer 280 hangs up the call at 1764.08
10
5
Customer 283 initiated a call at 1769.33
Customer 279 is done at 1771.84
Customer 283 is rooted to operator 2 at 1775.55
Customer 283 is assigned to the operator 2 at 1775.55
Customer 281 is rooted to operator 2 at 1780.7
Customer 283 is done at 1781.24
Customer 281 is assigned to the operator 2 at 1781.24
Customer 282 is rooted to operator 2 at 1783.82
10
5
Customer 284 initiated a call at 1785.21
Customer 281 is done at 1786.9
Customer 282 is assigned to the operator 2 at 1786.9
Customer 282 is done at 1788.35
Customer 284 is rooted to operator 1 at 1790.34
Customer 284 is assigned to the operator 1 at 1790.34
Customer 284 is done at 1797.7
10
5
Customer 285 initiated a call at 1809.92
Customer 285 is rooted to operator 2 at 1810.62
Customer 285 is assigned to the operator 2 at 1810.62
10
5
Customer 286 init

Customer 485 is assigned to the operator 2 at 2995.89
8
5
Customer 487 initiated a call at 2997.9
Customer 486 is rooted to operator 2 at 2998.09
Customer 482 is done at 2998.17
8
5
Customer 488 initiated a call at 2998.52
Customer 485 is done at 3000.55
Customer 486 is assigned to the operator 2 at 3000.55
8
5
Customer 489 initiated a call at 3001.06
Customer 489 is rooted to operator 1 at 3001.46
Customer 489 is assigned to the operator 1 at 3001.46
Customer 486 is done at 3001.77
8
5
Customer 490 initiated a call at 3003.36
8
5
Customer 491 initiated a call at 3005.02
Customer 490 is rooted to operator 2 at 3005.9
Customer 490 is assigned to the operator 2 at 3005.9
Customer 488 is rooted to operator 2 at 3006.04
8
5
Customer 492 initiated a call at 3011.54
Customer 491 hangs up the call at 3012.19
Customer 490 is done at 3012.63
Customer 488 is assigned to the operator 2 at 3012.63
Customer 487 is rooted to operator 2 at 3012.79
Customer 489 is done at 3012.95
Customer 492 is roote

Customer 763 is done at 4624.31
12
8
Customer 764 initiated a call at 4629.84
12
8
Customer 765 initiated a call at 4634.35
Customer 765 is rooted to operator 2 at 4635.76
Customer 765 is assigned to the operator 2 at 4635.76
Customer 765 is done at 4639.23
Customer 764 is rooted to operator 1 at 4640.32
Customer 764 is assigned to the operator 1 at 4640.32
12
8
Customer 766 initiated a call at 4640.32
12
8
Customer 767 initiated a call at 4641.4
Customer 766 is rooted to operator 2 at 4641.73
Customer 766 is assigned to the operator 2 at 4641.73
12
8
Customer 768 initiated a call at 4643.95
Customer 767 is rooted to operator 2 at 4645.22
Customer 768 is rooted to operator 2 at 4645.31
Customer 766 is done at 4646.79
Customer 767 is assigned to the operator 2 at 4646.79
12
8
Customer 769 initiated a call at 4647.58
Customer 769 is rooted to operator 2 at 4648.3
Customer 764 is done at 4649.68
Customer 767 is done at 4651.93
Customer 768 is assigned to the operator 2 at 4651.93
Customer